In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, roc_curve, roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier

from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.models import Sequential, Model, load_model
from tensorflow.python.keras.layers import BatchNormalization
from keras.layers import Input, Dense, Conv1D, Conv1DTranspose
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers

from scipy.interpolate import interp1d

In [ ]:
MITdf = pd.read_csv("/content/gdrive/MyDrive/ECG_Data/mitbih_train.csv", header = None, index_col= False)
MITdf_itp = pd.read_pickle("/content/gdrive/MyDrive/ECG_Data/interpolation.pkl")

In [ ]:
raw_data = MITdf.values
labels = raw_data[:,-1]
data = MITdf_itp.values


train_data, test_data, train_labels, test_labels = train_test_split(
    data, labels, test_size = 0.2, random_state = 21
)

min_val = tf.reduce_min(train_data)
max_val = tf.reduce_max(train_data)

train_data = (train_data - min_val) / (max_val - min_val)
test_data = (test_data - min_val) / (max_val - min_val)

train_data = tf.cast(train_data, tf.float32)
test_data = tf.cast(test_data, tf.float32)

In [ ]:
def knn_model(k, x_train, y_train):
  # regressor_k = KNeighborsRegressor(n_neighbors=k, weights="distance")
  # regressor_k.fit(x_train, y_train)
  knn = KNeighborsClassifier(n_neighbors=k)
  knn.fit(x_train, y_train)
  return knn

def print_model_sample(model, x_test, y_test, sample_num):
  output = model.predict(x_test)
  output = output.clip(min=0)
  output = pd.DataFrame(output, index=y_test.index.values)
  result = pd.merge(output, y_test, left_index = True, right_index = True, 
                    how='left')

  sample = result.sample(n=sample_num) 
  result.columns=["predicted future","real future"]
  sample.plot(figsize=(100,30), linewidth=3.0)
  print('R2 score : ' + str(r2_score(y_test,output))) #r2 score

In [ ]:
sample_ecg = test_data[3250]
sample_ecg_2d = sample_ecg.reshape(-1,1)
print(sample_ecg)
sample_ecg_label = test_labels[3250]
print(sample_ecg_label)


AttributeError: ignored

In [ ]:
knn_ecg = knn_model(10, train_data, train_labels)
print_model_sample(knn_ecg, test_data, test_labels, 300)

AttributeError: ignored